In [1]:
from os import path
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd

In [3]:
# account details
USERNAME = '_'
PASSWORD = '_'
EMAIL = '_'
ORG = 'UC Berkeley'

def register(username, password, email, org):
    url = 'https://api2.watttime.org/register'
    params = {'username': username,
              'password': password,
              'email': email,
              'org': org}
    rsp = requests.post(url, json=params)
    print(rsp.text)


def login(username, password):
    url = 'https://api2.watttime.org/login'
    try:
        rsp = requests.get(url, auth=HTTPBasicAuth(username, password))
    except BaseException as e:
        print('There was an error making your login request: {}'.format(e))
        return None

    try:
        token = rsp.json()['token']
    except BaseException:
        print('There was an error logging in. The message returned from the '
              'api is {}'.format(rsp.text))
        return None

    return token


def data(token, ba, starttime, endtime):
    url = 'https://api2.watttime.org/data'
    headers = {'Authorization': 'Bearer {}'.format(token)}
    params = {'ba': ba, 'starttime': starttime, 'endtime': endtime}

    rsp = requests.get(url, headers=headers, params=params)
    # print(rsp.text)  # uncomment to see raw response
    return rsp.json()


def index(token, ba):
    url = 'https://api2.watttime.org/index'
    headers = {'Authorization': 'Bearer {}'.format(token)}
    params = {'ba': ba}

    rsp = requests.get(url, headers=headers, params=params)
    # print(rsp.text)  # uncomment to see raw response
    return rsp.json()


def forecast(token, ba, starttime=None, endtime=None):
    url = 'https://api2.watttime.org/forecast'
    headers = {'Authorization': 'Bearer {}'.format(token)}
    params = {'ba': ba}
    if starttime:
        params.update({'starttime': starttime, 'endtime': endtime})

    rsp = requests.get(url, headers=headers, params=params)
    # print(rsp.text)  # uncomment to see raw response
    return rsp.json()


def historical(token, ba):
    url = 'https://api2.watttime.org/historical'
    headers = {'Authorization': 'Bearer {}'.format(token)}
    params = {'ba': ba}
    rsp = requests.get(url, headers=headers, params=params)
    cur_dir = path.dirname(path.realpath(__file__))
    file_path = path.join(cur_dir, '{}_historical.zip'.format(ba))
    with open(file_path, 'wb') as fp:
        fp.write(rsp.content)

    print('Wrote historical data for {} to {}'.format(ba, file_path))

token = login(USERNAME, PASSWORD)
if not token:
    print('You will need to fix your login credentials (username and password '
          'at the start of this file) before you can query other endpoints. '
          'Make sure that you have registered at least once by uncommenting '
          'the register(username, password, email, org) line near the bottom '
          'of this file.')
    exit()

In [5]:
# request details
BA = 'CAISO_NORTH'  # identify grid region

# starttime and endtime are optional, if ommited will return the latest value
START = '2021-01-01T00:00:00-8'  # UTC offset of 0 (PDT is -7, PST -8)
END = '2021-02-01T00:00:00-8'

realtime_index = index(token, BA)
#print(realtime_index)

# print('Please note: the following endpoints require a WattTime subscription')
# historical_moer = data(token, BA, START, END)
# print(historical_moer)

forecast_moer = forecast(token, BA)
#print(forecast_moer['forecast'])

# forecast_moer = forecast(token, BA, START, END)
# print(forecast_moer)

In [6]:
all_data = pd.DataFrame()
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
years = ['2017', '2018', '2019', '2020', '2021', '2022', '2023']
for year in years:
    for i in range(len(months) - 1):
        # starttime and endtime are optional, if ommited will return the latest value
        START = year + '-' + months[i] + '-01T00:00:00-8'  # UTC offset of 0 (PDT is -7, PST -8)
        END = year + '-' + months[i+1] + '-01T00:00:00-8'
        historical_moer = data(token, BA, START, END)
        all_data = all_data.append(pd.DataFrame(historical_moer))
all_data

,point_time,value,frequency,market,ba,datatype,version
0,2018-02-01T08:00:00.000Z,944.0,None,RTM,CAISO_NORTH,MOER,3.0
1,2018-02-01T07:55:00.000Z,1006.0,None,RTM,CAISO_NORTH,MOER,3.0
2,2018-02-01T07:50:00.000Z,981.0,None,RTM,CAISO_NORTH,MOER,3.0
3,2018-02-01T07:45:00.000Z,972.0,None,RTM,CAISO_NORTH,MOER,3.0
4,2018-02-01T07:40:00.000Z,972.0,None,RTM,CAISO_NORTH,MOER,3.0
...,...,...,...,...,...,...,...
3005,2022-10-01T08:20:00.000Z,938.0,300,RTM,CAISO_NORTH,MOER,3.0
3006,2022-10-01T08:15:00.000Z,934.0,300,RTM,CAISO_NORTH,MOER,3.0
3007,2022-10-01T08:10:00.000Z,931.0,300,RTM,CAISO_NORTH,MOER,3.0
3008,2022-10-01T08:05:00.000Z,934.0,300,RTM,CAISO_NORTH,MOER,3.0


In [8]:
for i in range(len(years) - 1):
    START = years[i] + '-12-01T00:00:00-8'  # UTC offset of 0 (PDT is -7, PST -8)
    END = years[i + 1] + '-01-01T00:00:00-8'    
    historical_moer = data(token, BA, START, END)
    all_data = all_data.append(pd.DataFrame(historical_moer))
all_data = all_data.sort_values("point_time")
all_data.to_csv("../Data/wattime_data.csv")
all_data

,point_time,value,frequency,market,ba,datatype,version
96,2018-01-01T00:00:00.000Z,1022.0,None,RTM,CAISO_NORTH,MOER,3.0
96,2018-01-01T00:00:00.000Z,1022.0,None,RTM,CAISO_NORTH,MOER,3.0
95,2018-01-01T00:05:00.000Z,1022.0,None,RTM,CAISO_NORTH,MOER,3.0
95,2018-01-01T00:05:00.000Z,1022.0,None,RTM,CAISO_NORTH,MOER,3.0
94,2018-01-01T00:10:00.000Z,1021.0,None,RTM,CAISO_NORTH,MOER,3.0
...,...,...,...,...,...,...,...
4,2022-10-11T18:25:00.000Z,857.0,300,RTM,CAISO_NORTH,MOER,3.0
3,2022-10-11T18:30:00.000Z,866.0,300,RTM,CAISO_NORTH,MOER,3.0
2,2022-10-11T18:35:00.000Z,842.0,300,RTM,CAISO_NORTH,MOER,3.0
1,2022-10-11T18:40:00.000Z,832.0,300,RTM,CAISO_NORTH,MOER,3.0


In [9]:
from datetime import datetime, timezone
from dateutil import tz
import pytz

In [11]:
df = all_data
df["date"] = [datetime.fromisoformat(s[:-1]) for s in df["point_time"]]
#df["date"] = [timestamp.tz_localize("America/Los_Angeles", ambiguous=0, nonexistent="shift_forward") for timestamp in df["date"]]
df["time"] = pd.DatetimeIndex(df['date']).time
df["year"] = [d.year for d in df["date"]]
df["month"] = [d.month for d in df["date"]]
df["day_of_month"] = pd.DatetimeIndex(df['date']).day
df["day_of_week"] = [datetime.weekday(d) for d in df["date"]]
df["day_of_week"] = df["day_of_week"].replace({0 : "Mon", 1 : "Tue", 2 : "Wed", 3 : "Thurs", 4 : "Fri", 5 : "Sat", 6 : "Sun"})
df["hour"] = [d.hour for d in df["time"]]
df["minute"] = [d.minute for d in df["time"]]
df = df.rename(columns = {"value" : "MOER"})
df = df.drop("point_time", axis=1)
data18 = df[df['year'] == 2018]
data18 = data18.sort_values("date")
data18 = data18[data18["minute"] % 15 == 0]
data18 = data18.drop(["frequency", "market", "ba", "datatype", "version", "time", "month", "day_of_month", "day_of_week", "minute", "hour"], axis= 1)
data18


,MOER,date,year
96,1022.0,2018-01-01 00:00:00,2018
96,1022.0,2018-01-01 00:00:00,2018
93,1021.0,2018-01-01 00:15:00,2018
93,1021.0,2018-01-01 00:15:00,2018
90,1018.0,2018-01-01 00:30:00,2018
...,...,...,...
105,1017.0,2018-12-31 23:15:00,2018
102,1028.0,2018-12-31 23:30:00,2018
102,1028.0,2018-12-31 23:30:00,2018
99,1120.0,2018-12-31 23:45:00,2018


In [13]:
ca_data = pd.read_csv("../Data/15minute_data_california/15minute_data_california.csv")
ca_data = ca_data[["dataid", "local_15min", "leg1v", "leg2v", "grid"]]
ca_data["date"] = pd.to_datetime(ca_data["local_15min"].str[:-3], format='%Y-%m-%d %H:%M:%S')
ca_data["year"] = [d.year for d in ca_data["date"]]
ca_data["day of week"] = [d.weekday() for d in ca_data["date"]]
ca_data["hour"] = [d.hour for d in ca_data["date"]]
ca_data = ca_data.sort_values("date")
ca18 = ca_data[ca_data["year"] == 2018]
ca18 = ca18.drop(["hour", "day of week", "leg2v", "dataid", "local_15min"], axis= 1)
ca18 = ca18.groupby("date").mean()
ca18

,leg1v,grid,year
date,,,
2018-01-01 00:00:00,120.209333,0.419000,2018
2018-01-01 00:15:00,120.416333,0.498000,2018
2018-01-01 00:30:00,120.478333,0.536000,2018
2018-01-01 00:45:00,120.237333,0.620000,2018
2018-01-01 01:00:00,120.376333,0.551333,2018
...,...,...,...
2018-12-31 22:45:00,120.432333,0.617667,2018
2018-12-31 23:00:00,119.971333,0.550667,2018
2018-12-31 23:15:00,120.057000,0.542667,2018


In [14]:
all18 = data18
all18.merge(ca18, on= "date")

,MOER,date,year_x,leg1v,grid,year_y
0,1022.0,2018-01-01 00:00:00,2018,120.209333,0.419000,2018
1,1022.0,2018-01-01 00:00:00,2018,120.209333,0.419000,2018
2,1021.0,2018-01-01 00:15:00,2018,120.416333,0.498000,2018
3,1021.0,2018-01-01 00:15:00,2018,120.416333,0.498000,2018
4,1018.0,2018-01-01 00:30:00,2018,120.478333,0.536000,2018
...,...,...,...,...,...,...
37993,1017.0,2018-12-31 23:15:00,2018,120.057000,0.542667,2018
37994,1028.0,2018-12-31 23:30:00,2018,120.116667,0.519000,2018
37995,1028.0,2018-12-31 23:30:00,2018,120.116667,0.519000,2018
37996,1120.0,2018-12-31 23:45:00,2018,120.129667,0.456000,2018
